Run the following to obtain the word list:

```
curl -LO https://github.com/danakt/russian-words/raw/master/russian.txt
iconv -f cp1251 -t utf8 russian.txt | awk '{print tolower($0)}' > russian-utf8.txt
```

In [1]:
import codecs

with open('russian-utf8.txt', 'r') as f:
    russian_words = f.read().splitlines()

ciphertext_file = 'mono_var08.KR'
space_sub = 'Ы'
with codecs.open(ciphertext_file, 'r', 'cp1251') as f:
    ciphertext = f.read()

ciphertext_words = list(set(ciphertext.split(space_sub)))

In [2]:
def word_to_pattern(word):
    pattern = []
    letters = []
    for l in word:
        if l in letters:
            pattern.append(letters.index(l))
        else:
            letters.append(l)
            pattern.append(len(letters) - 1)
    return pattern

dict_patterns = [(w, word_to_pattern(w)) for w in russian_words]

In [3]:
def match_ciphertext_word(ciph_word):
    matches = []
    ciph_pattern = word_to_pattern(ciph_word)
    for dict_word, dict_pattern in dict_patterns:
        if ciph_pattern == dict_pattern:
            subs = dict(zip(ciph_word, dict_word))
            matches.append((dict_word, subs))
    return matches

def word_subs(ciph_word):
    subs = {}
    for dict_word, dict_subs in match_ciphertext_word(ciph_word):
        for ciph, deciph in dict_subs.items():
            if ciph in subs:
                subs[ciph].add(deciph)
            else:
                subs[ciph] = set([deciph])
    return subs

The following codes prints out a list of possible letter substitutions, in the order of most confident (less possibilities) -> least confident (more possibilities). Using it, I cracked the cypher by hand in a couple of minutes.

In [4]:
global_subs = {}
for w in ciphertext_words:
    for ciph, deciphs in word_subs(w).items():
        if ciph in global_subs:
            global_subs[ciph] = global_subs[ciph].intersection(deciphs)
        else:
            global_subs[ciph] = deciphs

letter_subs = sorted(global_subs.items(), key=lambda subs: len(subs[1]))
for ciph, deciphs in letter_subs:
    print(f'{ciph} -> {deciphs}')

Щ -> {'п'}
Ц -> {'т'}
З -> {'н'}
Ф -> {'с'}
Л -> {'е'}
Т -> {'д'}
Г -> {'о'}
М -> {'и'}
Ж -> {'ш'}
К -> {'я'}
О -> {'а', 'е'}
Ш -> {'м', 'х'}
Р -> {'в', 'к', 'л', 'т'}
Ч -> {'о', 'у', 'ы', 'а'}
П -> {'д', 'п', 'р', 'б', 'з', 'ш'}
Ю -> {'н', 'с', 'к', 'р', 'ч', 'л', 'т', 'я', 'и'}
Ь -> {'н', 'д', 'ш', 'с', 'щ', 'в', 'ц', 'ч', 'х', 'т'}
Х -> {'д', 'м', 'в', 'к', 'р', 'о', 'у', 'а', 'л', 'и', 'т', 'я', 'е', 'ж'}
Й -> {'н', 'д', 'с', 'п', 'м', 'в', 'к', 'р', 'б', 'з', 'х', 'а', 'л', 'т'}
Э -> {'н', 'д', 'с', 'п', 'м', 'в', 'р', 'б', 'з', 'ч', 'о', 'г', 'а', 'л', 'т', 'ф'}
И -> {'д', 'н', 'с', 'щ', 'в', 'к', 'р', 'ю', 'у', 'а', 'г', 'и', 'ш', 'я', 'й', 'е', 'ж'}
Д -> {'д', 'ы', 'м', 'в', 'к', 'ь', 'з', 'ц', 'ю', 'о', 'у', 'а', 'л', 'и', 'т', 'я', 'й', 'е'}
Н -> {'ё', 'в', 'ь', 'т', 'ж', 'м', 'х', 'а', 'б', 'ц', 'л', 'и', 'ш', 'н', 'с', 'к', 'р', 'у', 'е'}
В -> {'щ', 'в', 'з', 'ю', 'т', 'ж', 'д', 'м', 'х', 'а', 'п', 'б', 'л', 'и', 'ш', 'н', 'с', 'к', 'р', 'ч', 'о', 'у', 'г', 'е'}
Ъ -> {'щ', 